# 2-PandasDataframe

This notebook demonstrates how to read and process a tabular datafile with the [Pandas](https://pandas.pydata.org/) dataframe library. Pandas dataframes are limited to run on a single core and data must fit into memory.

Author: Peter W. Rose (pwrose@ucsd.edu)

In [1]:
import os
import pandas as pd
import time

If LOCAL_SCRATCH_DIR environment variable is not set, this notebook accesses the ../data directory for temporary files.

In [2]:
DATA_DIR = os.getenv("LOCAL_SCRATCH_DIR", default="../data")

### Setup Benchmark

The ```file_format``` parameter is used for benchmarking different file formats. 
This Cell [3] has been [parameterized](https://papermill.readthedocs.io/en/latest/usage-parameterize.html) as input parameters for [papermill](https://papermill.readthedocs.io/en/latest/index.html).

In [3]:
file_format = "csv"

In [4]:
start = time.time()

### Read Data

In [5]:
column_names = ["GeneID", "Symbol", "Synonyms", "description", "type_of_gene", "#tax_id", "chromosome"]

if file_format == "csv":
    filename = os.path.join(DATA_DIR, "gene_info.tsv")
    genes = pd.read_csv(filename, usecols=column_names, dtype=str, sep="\t")
    #genes = pd.read_csv(filename, usecols=column_names, dtype={'tax_id': int, 'GeneID': int, 'type_of_gene': 'category'}, sep="\t")    
elif file_format == "parquet":
    filename = os.path.join(DATA_DIR, "gene_info.parquet")
    genes = pd.read_parquet(filename, columns=column_names)
else:
    print("invalid file format")
    
print("Filename:", filename)
file_size = os.path.getsize(filename)
print(f"File Size: {file_size/1E9:.1f} GB")
    
genes.rename(columns={"#tax_id": "tax_id"}, inplace=True)

Filename: ../data/gene_info.tsv
File Size: 5.4 GB


In [6]:
# print(f"Total memory: {genes.memory_usage(deep=True).sum()/1E9} GB")
# Total memory: 17.979084252 GB

### Process Data

In [7]:
genes.query("type_of_gene == 'protein-coding'", inplace=True)

In [8]:
genes.head()

,tax_id,GeneID,Symbol,Synonyms,chromosome,description,type_of_gene
6,24,72485293,dnaA,MZ182_00005,-,chromosomal replication initiator protein DnaA,protein-coding
7,24,72485294,dnaN,MZ182_00010,-,DNA polymerase III subunit beta,protein-coding
8,24,72485295,recF,MZ182_00015,-,DNA replication/repair protein RecF,protein-coding
9,24,72485296,gyrB,MZ182_00020,-,DNA topoisomerase (ATP-hydrolyzing) subunit B,protein-coding
10,24,72485297,MZ182_RS00025,MZ182_00025,-,HDOD domain-containing protein,protein-coding


In [9]:
groups = genes.groupby(["tax_id"]).size().reset_index(name="count")
groups = groups.sort_values("count", ascending=False)

### Display Results

#### Number of human protein-coding genes (tax_id = 9606)

In [10]:
groups.query("tax_id == '9606'")

,tax_id,count
36154,9606,20598


#### Top 5 organisms with the most protein-coding genes

In [11]:
groups.head()

,tax_id,count
27954,4565,104035
25514,3708,90975
35480,90675,82686
35974,94328,68154
25227,3635,67632


In [12]:
end = time.time()

In [13]:
print(f"Pandas: {end - start:.1f} sec.")

Pandas: 115.4 sec.
